# Settings

In [63]:
import requests
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


In [64]:
pd.set_option('display.max_columns', 30)

# Loading Data

In [65]:
df_clubes = pd.read_csv("data/clubes.csv", index_col=0)
df_clubes

,id,nome,abreviacao,slug,apelido,nome_fantasia
0,1,OUT,OUT,NaN,NaN,Outros
1,1349,IPA,IPA,NaN,NaN,Ipatinga
2,1371,CUI,CUI,NaN,NaN,Cuiabá
3,1390,ICA,ICA,NaN,NaN,Icasa
4,2190,OES,OES,NaN,NaN,Oeste
...,...,...,...,...,...,...
63,364,REM,REM,NaN,NaN,Remo
64,373,ACG,ACG,NaN,NaN,Atlético-GO
65,375,VIL,VIL,NaN,NaN,Vila Nova
66,386,SAO,SAO,NaN,NaN,São Caetano


In [66]:
df_rodadas = pd.read_csv("data/partidas.csv", index_col=0)
df_rodadas

,local,mandante_id,visitante_id,placar_mandante,placar_visitante,posicao_mandante,posicao_visitante,aproveitamento_mandante,aproveitamento_visitante,rodada_id
0,Morumbis,276,292,0.0,0.0,7,20,"['v', 'v', 'e', 'v', 'd']","['e', 'v', 'e', 'd', 'e']",1
1,Mineirão,283,2305,2.0,1.0,2,6,"['v', 'd', 'e', 'v', 'v']","['v', 'd', 'e', 'v', 'e']",1
2,Arena do Grêmio,284,282,2.0,1.0,14,12,"['d', 'd', 'v', 'e', 'e']","['v', 'e', 'd', 'd', 'e']",1
3,Castelão (CE),356,266,2.0,0.0,19,9,"['e', 'd', 'd', 'd', 'e']","['v', 'e', 'v', 'd', 'e']",1
4,Alfredo Jaconi,286,287,2.0,0.0,16,18,"['v', 'e', 'v', 'd', 'v']","['e', 'd', 'e', 'd', 'e']",1
...,...,...,...,...,...,...,...,...,...,...
205,Castelão (CE),356,2305,0.0,1.0,19,6,"['e', 'd', 'd', 'd', 'e']","['v', 'd', 'e', 'v', 'e']",21
206,Alfredo Jaconi,286,263,1.0,3.0,16,5,"['v', 'e', 'v', 'd', 'v']","['d', 'v', 'd', 'v', 'v']",21
207,Morumbis,276,282,2.0,0.0,7,12,"['v', 'v', 'e', 'v', 'd']","['v', 'e', 'd', 'd', 'e']",21
208,Allianz Parque,275,292,3.0,0.0,3,20,"['e', 'v', 'v', 'v', 'e']","['e', 'v', 'e', 'd', 'e']",21


In [67]:
df_posicoes = pd.read_csv("data/posicoes.csv", index_col=0)
df_posicoes

,id,nome,abreviacao
0,1,Goleiro,gol
1,2,Lateral,lat
2,3,Zagueiro,zag
3,4,Meia,mei
4,5,Atacante,ata
5,6,Técnico,tec


In [68]:
df_scouts_values = {
    'id': range(1, 12),
    'nome': [
        'Gol', 
        'Assistência', 
        'Finalização na trave',
        'Finalização defendida', 
        'Finalização para fora',
        'Falta sofrida', 
        'Pênalti sofrido',
        'Pênalti perdido para fora', 
        'Pênalti perdido com defesa do goleiro',
        'Pênalti perdido na trave', 
        'Impedimento'
    ],
    'abreviacao': [
        'G', 'A', 'FT', 'FD', 'FF', 'FS', 'PS', 'PPF', 'PPD', 'PPT', 'I'
    ],
    'pontos': [
        8.0, 5.0, 3.0, 1.2, 0.8, 0.5, 1.0, -3.2, -2.8, -1.0, -0.1
    ]
}

df_scouts = pd.DataFrame(df_scouts_values)
df_scouts

,id,nome,abreviacao,pontos
0,1,Gol,G,8.0
1,2,Assistência,A,5.0
2,3,Finalização na trave,FT,3.0
3,4,Finalização defendida,FD,1.2
4,5,Finalização para fora,FF,0.8
5,6,Falta sofrida,FS,0.5
6,7,Pênalti sofrido,PS,1.0
7,8,Pênalti perdido para fora,PPF,-3.2
8,9,Pênalti perdido com defesa do goleiro,PPD,-2.8
9,10,Pênalti perdido na trave,PPT,-1.0


In [69]:
df_partidas = pd.read_csv("data/rodadas.csv", index_col=0)
df_partidas.head()

,rodada_id,nome_rodada,inicio,fim
0,1,Rodada 1,2025-03-29 18:30:00,2025-03-31 20:00:00
1,2,Rodada 2,2025-04-05 18:30:00,2025-04-06 20:30:00
2,3,Rodada 3,2025-04-12 16:00:00,2025-04-13 20:30:00
3,4,Rodada 4,2025-04-16 18:30:00,2025-04-17 21:30:00
4,5,Rodada 5,2025-04-19 16:00:00,2025-04-21 20:00:00


In [70]:
df_status = pd.read_csv("data/status.csv", index_col=0)
df_status

,id,nome
0,2,Dúvida
1,3,Suspenso
2,5,Contundido
3,6,Nulo
4,7,Provável


In [71]:
partidas_totais = pd.read_csv("data/partidas_totais.csv", index_col=0)
partidas_totais.head()

,mandante_id,visitante_id,rodada_id
0,262,285,1
1,267,277,1
2,275,263,1
3,276,292,1
4,280,354,1


In [72]:
atleta_pontuacao = pd.read_csv("data/jogadores.csv", index_col=0)
atleta_pontuacao.head()

,id,apelido,pontuacao,posicao_id,clube_id,scouts,entrou,rodada_id
0,100652,Yuri Alberto,2.9,5,264,"{'FD': 2, 'FS': 1}",True,1
1,100742,Matheus Henrique,-1.9,4,283,"{'CA': 1, 'FC': 3}",True,1
2,100846,Lucas Esteves,2.8,2,284,"{'DS': 1, 'FF': 1, 'FS': 1}",True,1
3,100898,Vitinho,10.7,2,263,"{'DS': 4, 'FC': 1, 'SG': 1}",True,1
4,100969,Ronald Lopes,-1.1,4,287,"{'CA': 1, 'FC': 2, 'FS': 1}",True,1


# Adjusting partidas

In [73]:
partidas_historico = df_rodadas.drop(columns=["local", "rodada_id"])
partidas_historico

,mandante_id,visitante_id,placar_mandante,placar_visitante,posicao_mandante,posicao_visitante,aproveitamento_mandante,aproveitamento_visitante
0,276,292,0.0,0.0,7,20,"['v', 'v', 'e', 'v', 'd']","['e', 'v', 'e', 'd', 'e']"
1,283,2305,2.0,1.0,2,6,"['v', 'd', 'e', 'v', 'v']","['v', 'd', 'e', 'v', 'e']"
2,284,282,2.0,1.0,14,12,"['d', 'd', 'v', 'e', 'e']","['v', 'e', 'd', 'd', 'e']"
3,356,266,2.0,0.0,19,9,"['e', 'd', 'd', 'd', 'e']","['v', 'e', 'v', 'd', 'e']"
4,286,287,2.0,0.0,16,18,"['v', 'e', 'v', 'd', 'v']","['e', 'd', 'e', 'd', 'e']"
...,...,...,...,...,...,...,...,...
205,356,2305,0.0,1.0,19,6,"['e', 'd', 'd', 'd', 'e']","['v', 'd', 'e', 'v', 'e']"
206,286,263,1.0,3.0,16,5,"['v', 'e', 'v', 'd', 'v']","['d', 'v', 'd', 'v', 'v']"
207,276,282,2.0,0.0,7,12,"['v', 'v', 'e', 'v', 'd']","['v', 'e', 'd', 'd', 'e']"
208,275,292,3.0,0.0,3,20,"['e', 'v', 'v', 'v', 'e']","['e', 'v', 'e', 'd', 'e']"


In [74]:
partidas_historico["key"] = partidas_historico["mandante_id"].astype(str) + "_" + partidas_historico["visitante_id"].astype(str)
partidas_historico


,mandante_id,visitante_id,placar_mandante,placar_visitante,posicao_mandante,posicao_visitante,aproveitamento_mandante,aproveitamento_visitante,key
0,276,292,0.0,0.0,7,20,"['v', 'v', 'e', 'v', 'd']","['e', 'v', 'e', 'd', 'e']",276_292
1,283,2305,2.0,1.0,2,6,"['v', 'd', 'e', 'v', 'v']","['v', 'd', 'e', 'v', 'e']",283_2305
2,284,282,2.0,1.0,14,12,"['d', 'd', 'v', 'e', 'e']","['v', 'e', 'd', 'd', 'e']",284_282
3,356,266,2.0,0.0,19,9,"['e', 'd', 'd', 'd', 'e']","['v', 'e', 'v', 'd', 'e']",356_266
4,286,287,2.0,0.0,16,18,"['v', 'e', 'v', 'd', 'v']","['e', 'd', 'e', 'd', 'e']",286_287
...,...,...,...,...,...,...,...,...,...
205,356,2305,0.0,1.0,19,6,"['e', 'd', 'd', 'd', 'e']","['v', 'd', 'e', 'v', 'e']",356_2305
206,286,263,1.0,3.0,16,5,"['v', 'e', 'v', 'd', 'v']","['d', 'v', 'd', 'v', 'v']",286_263
207,276,282,2.0,0.0,7,12,"['v', 'v', 'e', 'v', 'd']","['v', 'e', 'd', 'd', 'e']",276_282
208,275,292,3.0,0.0,3,20,"['e', 'v', 'v', 'v', 'e']","['e', 'v', 'e', 'd', 'e']",275_292


In [75]:
partidas_totais["key"] = partidas_totais["mandante_id"].astype(str) + "_" + partidas_totais["visitante_id"].astype(str)
partidas_totais

,mandante_id,visitante_id,rodada_id,key
0,262,285,1,262_285
1,267,277,1,267_277
2,275,263,1,275_263
3,276,292,1,276_292
4,280,354,1,280_354
...,...,...,...,...
375,282,267,38,282_267
376,285,280,38,285_280
377,287,276,38,287_276
378,354,275,38,354_275


In [76]:
dados_partida = pd.merge(partidas_totais, partidas_historico, on="key")
dados_partida

,mandante_id_x,visitante_id_x,rodada_id,key,mandante_id_y,visitante_id_y,placar_mandante,placar_visitante,posicao_mandante,posicao_visitante,aproveitamento_mandante,aproveitamento_visitante
0,262,285,1,262_285,262,285,1.0,1.0,1,13,"['e', 'v', 'v', 'v', 'e']","['d', 'v', 'd', 'd', 'e']"
1,267,277,1,267_277,267,277,2.0,1.0,17,15,"['e', 'v', 'd', 'd', 'e']","['v', 'v', 'd', 'd', 'e']"
2,275,263,1,275_263,275,263,0.0,0.0,3,5,"['e', 'v', 'v', 'v', 'e']","['d', 'v', 'd', 'v', 'v']"
3,276,292,1,276_292,276,292,0.0,0.0,7,20,"['v', 'v', 'e', 'v', 'd']","['e', 'v', 'e', 'd', 'e']"
4,280,354,1,280_354,280,354,2.0,2.0,8,10,"['d', 'd', 'd', 'v', 'd']","['e', 'd', 'v', 'e', 'd']"
...,...,...,...,...,...,...,...,...,...,...,...,...
205,283,285,21,283_285,283,285,2.0,1.0,2,13,"['v', 'd', 'e', 'v', 'v']","['d', 'v', 'd', 'd', 'e']"
206,284,354,21,284_354,284,354,0.0,0.0,14,10,"['d', 'd', 'v', 'e', 'e']","['e', 'd', 'v', 'e', 'd']"
207,265,277,21,265_277,265,277,2.0,0.0,4,15,"['e', 'e', 'v', 'v', 'e']","['v', 'v', 'd', 'd', 'e']"
208,356,2305,21,356_2305,356,2305,0.0,1.0,19,6,"['e', 'd', 'd', 'd', 'e']","['v', 'd', 'e', 'v', 'e']"


In [77]:
dados_partida = dados_partida.drop(columns=["key", "mandante_id_x", "visitante_id_x"])

In [78]:
dados_partida["resultado"] = dados_partida.apply(lambda x: 1 if x["placar_mandante"] > x["placar_visitante"] else (0 if x["placar_mandante"] == x["placar_visitante"] else -1), axis=1)
dados_partida

,rodada_id,mandante_id_y,visitante_id_y,placar_mandante,placar_visitante,posicao_mandante,posicao_visitante,aproveitamento_mandante,aproveitamento_visitante,resultado
0,1,262,285,1.0,1.0,1,13,"['e', 'v', 'v', 'v', 'e']","['d', 'v', 'd', 'd', 'e']",0
1,1,267,277,2.0,1.0,17,15,"['e', 'v', 'd', 'd', 'e']","['v', 'v', 'd', 'd', 'e']",1
2,1,275,263,0.0,0.0,3,5,"['e', 'v', 'v', 'v', 'e']","['d', 'v', 'd', 'v', 'v']",0
3,1,276,292,0.0,0.0,7,20,"['v', 'v', 'e', 'v', 'd']","['e', 'v', 'e', 'd', 'e']",0
4,1,280,354,2.0,2.0,8,10,"['d', 'd', 'd', 'v', 'd']","['e', 'd', 'v', 'e', 'd']",0
...,...,...,...,...,...,...,...,...,...,...
205,21,283,285,2.0,1.0,2,13,"['v', 'd', 'e', 'v', 'v']","['d', 'v', 'd', 'd', 'e']",1
206,21,284,354,0.0,0.0,14,10,"['d', 'd', 'v', 'e', 'e']","['e', 'd', 'v', 'e', 'd']",0
207,21,265,277,2.0,0.0,4,15,"['e', 'e', 'v', 'v', 'e']","['v', 'v', 'd', 'd', 'e']",1
208,21,356,2305,0.0,1.0,19,6,"['e', 'd', 'd', 'd', 'e']","['v', 'd', 'e', 'v', 'e']",-1


In [79]:
def getChanceToWin(pos1, pos2, option):
    chances = {pos1: 0, pos2: 0}

    for k, v in chances.items():
        if k <= 10: 
            chances[k] = 3 # Ponto por estar no meio da tabela
        elif k <= 15:
            chances[k] = 2 # Ponto por estar na 2 metade da tabela
        else: 
            chances[k] = 1 # Ponto por estar entre os 
    
    chances[pos1] += 1 # Ganha ponto por ser mandante

    if option == 1:
        dif = chances[pos1] - chances[pos2]
    else:
        dif = chances[pos2] - chances[pos1]

    if dif >= 1:
        return 1
    elif dif <= -1:
        return -1
    else:
        return 0
    

dados_partida["dificuldade_mandante"] = dados_partida.apply(lambda df: getChanceToWin(df["posicao_mandante"], df["posicao_visitante"], 1), axis=1)
dados_partida["dificuldade_visitante"] = dados_partida.apply(lambda df: getChanceToWin(df["posicao_mandante"], df["posicao_visitante"], 2), axis=1)

dados_partida


,rodada_id,mandante_id_y,visitante_id_y,placar_mandante,placar_visitante,posicao_mandante,posicao_visitante,aproveitamento_mandante,aproveitamento_visitante,resultado,dificuldade_mandante,dificuldade_visitante
0,1,262,285,1.0,1.0,1,13,"['e', 'v', 'v', 'v', 'e']","['d', 'v', 'd', 'd', 'e']",0,1,-1
1,1,267,277,2.0,1.0,17,15,"['e', 'v', 'd', 'd', 'e']","['v', 'v', 'd', 'd', 'e']",1,0,0
2,1,275,263,0.0,0.0,3,5,"['e', 'v', 'v', 'v', 'e']","['d', 'v', 'd', 'v', 'v']",0,1,-1
3,1,276,292,0.0,0.0,7,20,"['v', 'v', 'e', 'v', 'd']","['e', 'v', 'e', 'd', 'e']",0,1,-1
4,1,280,354,2.0,2.0,8,10,"['d', 'd', 'd', 'v', 'd']","['e', 'd', 'v', 'e', 'd']",0,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
205,21,283,285,2.0,1.0,2,13,"['v', 'd', 'e', 'v', 'v']","['d', 'v', 'd', 'd', 'e']",1,1,-1
206,21,284,354,0.0,0.0,14,10,"['d', 'd', 'v', 'e', 'e']","['e', 'd', 'v', 'e', 'd']",0,0,0
207,21,265,277,2.0,0.0,4,15,"['e', 'e', 'v', 'v', 'e']","['v', 'v', 'd', 'd', 'e']",1,1,-1
208,21,356,2305,0.0,1.0,19,6,"['e', 'd', 'd', 'd', 'e']","['v', 'd', 'e', 'v', 'e']",-1,-1,1


In [80]:
mandante_df = dados_partida.rename(columns={
    'mandante_id_y': 'clube_id',
    'visitante_id_y': 'rival_id',
    'placar_mandante': 'placar',
    'posicao_mandante': 'posicao',
    'aproveitamento_mandante': 'aproveitamento',
    'dificuldade_mandante': 'dificuldade'
})[['clube_id', 'rival_id', 'rodada_id', 'placar', 'posicao', 'aproveitamento', 'dificuldade']]

mandante_df.head()

,clube_id,rival_id,rodada_id,placar,posicao,aproveitamento,dificuldade
0,262,285,1,1.0,1,"['e', 'v', 'v', 'v', 'e']",1
1,267,277,1,2.0,17,"['e', 'v', 'd', 'd', 'e']",0
2,275,263,1,0.0,3,"['e', 'v', 'v', 'v', 'e']",1
3,276,292,1,0.0,7,"['v', 'v', 'e', 'v', 'd']",1
4,280,354,1,2.0,8,"['d', 'd', 'd', 'v', 'd']",1


In [81]:
visitante_df = dados_partida.rename(columns={
    'visitante_id_y': 'clube_id',
    'mandante_id_y': 'rival_id',
    'placar_visitante': 'placar',
    'posicao_visitante': 'posicao',
    'aproveitamento_visitante': 'aproveitamento',
    'dificuldade_visitante': 'dificuldade'
})[['clube_id', 'rival_id', 'rodada_id', 'placar', 'posicao', 'aproveitamento', 'dificuldade']]
visitante_df.head()

,clube_id,rival_id,rodada_id,placar,posicao,aproveitamento,dificuldade
0,285,262,1,1.0,13,"['d', 'v', 'd', 'd', 'e']",-1
1,277,267,1,1.0,15,"['v', 'v', 'd', 'd', 'e']",0
2,263,275,1,0.0,5,"['d', 'v', 'd', 'v', 'v']",-1
3,292,276,1,0.0,20,"['e', 'v', 'e', 'd', 'e']",-1
4,354,280,1,2.0,10,"['e', 'd', 'v', 'e', 'd']",-1


In [82]:

partidas_formatadas = pd.concat([mandante_df, visitante_df], ignore_index=True)
partidas_formatadas.head(20)

,clube_id,rival_id,rodada_id,placar,posicao,aproveitamento,dificuldade
0,262,285,1,1.0,1,"['e', 'v', 'v', 'v', 'e']",1
1,267,277,1,2.0,17,"['e', 'v', 'd', 'd', 'e']",0
2,275,263,1,0.0,3,"['e', 'v', 'v', 'v', 'e']",1
3,276,292,1,0.0,7,"['v', 'v', 'e', 'v', 'd']",1
4,280,354,1,2.0,8,"['d', 'd', 'd', 'v', 'd']",1
5,283,2305,1,2.0,2,"['v', 'd', 'e', 'v', 'v']",1
6,284,282,1,2.0,14,"['d', 'd', 'v', 'e', 'e']",1
7,265,264,1,1.0,4,"['e', 'e', 'v', 'v', 'e']",1
8,356,266,1,2.0,19,"['e', 'd', 'd', 'd', 'e']",-1
9,286,287,1,2.0,16,"['v', 'e', 'v', 'd', 'v']",1


In [98]:
def convertAproveitamento(item):

    item = eval(item)

    sum = 0
    for i in item:
        if i == "d":
            sum -= 1
        elif i == "v":
            sum += 1

    if sum >= 3:
        return 1
    elif sum <= -3:
        return -1
    else:
       return 0

partidas_formatadas["aproveitamento_5rounds"]= partidas_formatadas.apply(lambda df: convertAproveitamento(df["aproveitamento"]), axis=1)

partidas_formatadas

,clube_id,rival_id,rodada_id,placar,posicao,aproveitamento,dificuldade,aproveitamento_5rounds,aproveitamento_3rounds
0,262,285,1,1.0,1,"['e', 'v', 'v', 'v', 'e']",1,1,2
1,267,277,1,2.0,17,"['e', 'v', 'd', 'd', 'e']",0,0,-2
2,275,263,1,0.0,3,"['e', 'v', 'v', 'v', 'e']",1,1,2
3,276,292,1,0.0,7,"['v', 'v', 'e', 'v', 'd']",1,0,0
4,280,354,1,2.0,8,"['d', 'd', 'd', 'v', 'd']",1,-1,-1
...,...,...,...,...,...,...,...,...,...
415,285,283,21,1.0,13,"['d', 'v', 'd', 'd', 'e']",-1,0,-2
416,354,284,21,0.0,10,"['e', 'd', 'v', 'e', 'd']",0,0,0
417,277,265,21,0.0,15,"['v', 'v', 'd', 'd', 'e']",-1,0,-2
418,2305,356,21,1.0,6,"['v', 'd', 'e', 'v', 'e']",1,0,1


In [99]:
def convertAproveitamentoInValue2(item):
    sum = 0
    item = eval(item)
    for i in item[2:]:
        if i == "d":
            sum -= 1
        elif i == "v":
            sum += 1

    if sum >= 2:
        return 1
    elif sum <= -2:
        return -1
    else:
        return 0

partidas_formatadas["aproveitamento_3rounds"] = partidas_formatadas.apply(lambda df: convertAproveitamentoInValue2(df["aproveitamento"]), axis=1)

partidas_formatadas

,clube_id,rival_id,rodada_id,placar,posicao,aproveitamento,dificuldade,aproveitamento_5rounds,aproveitamento_3rounds
0,262,285,1,1.0,1,"['e', 'v', 'v', 'v', 'e']",1,1,1
1,267,277,1,2.0,17,"['e', 'v', 'd', 'd', 'e']",0,0,-1
2,275,263,1,0.0,3,"['e', 'v', 'v', 'v', 'e']",1,1,1
3,276,292,1,0.0,7,"['v', 'v', 'e', 'v', 'd']",1,0,0
4,280,354,1,2.0,8,"['d', 'd', 'd', 'v', 'd']",1,-1,0
...,...,...,...,...,...,...,...,...,...
415,285,283,21,1.0,13,"['d', 'v', 'd', 'd', 'e']",-1,0,-1
416,354,284,21,0.0,10,"['e', 'd', 'v', 'e', 'd']",0,0,0
417,277,265,21,0.0,15,"['v', 'v', 'd', 'd', 'e']",-1,0,-1
418,2305,356,21,1.0,6,"['v', 'd', 'e', 'v', 'e']",1,0,0


# Adjusting atleta_pontuacao

In [47]:
#import json


#scouts_df = atleta_pontuacao['scouts'].apply(lambda x: json.loads(x) if isinstance(x, str) else {})
#scouts_df = pd.json_normalize(scouts_df)


#dados = pd.concat([atleta_pontuacao.drop(columns=['scouts']), scouts_df], axis=1)

In [100]:
dados = atleta_pontuacao.drop(columns=["scouts"])
dados

,id,apelido,pontuacao,posicao_id,clube_id,entrou,rodada_id
0,100652,Yuri Alberto,2.9,5,264,True,1
1,100742,Matheus Henrique,-1.9,4,283,True,1
2,100846,Lucas Esteves,2.8,2,284,True,1
3,100898,Vitinho,10.7,2,263,True,1
4,100969,Ronald Lopes,-1.1,4,287,True,1
...,...,...,...,...,...,...,...
6405,99248,Bareiro,-3.3,5,356,True,21
6406,99267,Pochettino,3.2,4,356,True,21
6407,99274,Alexander Barboza,0.2,3,263,True,21
6408,99552,Rossi,6.3,1,262,True,21


In [101]:
dados = dados.merge(df_posicoes, left_on='posicao_id', right_on="id", how='left')
dados = dados.merge(df_clubes, left_on='clube_id', right_on="id", how='left')
dados.merge
dados.head()

,id_x,apelido_x,pontuacao,posicao_id,clube_id,entrou,rodada_id,id_y,nome_x,abreviacao_x,id,nome_y,abreviacao_y,slug,apelido_y,nome_fantasia
0,100652,Yuri Alberto,2.9,5,264,True,1,5,Atacante,ata,264,COR,COR,corinthians,Timão,Corinthians
1,100742,Matheus Henrique,-1.9,4,283,True,1,4,Meia,mei,283,CRU,CRU,cruzeiro,Raposa,Cruzeiro
2,100846,Lucas Esteves,2.8,2,284,True,1,2,Lateral,lat,284,GRE,GRE,gremio,Imortal,Grêmio
3,100898,Vitinho,10.7,2,263,True,1,2,Lateral,lat,263,BOT,BOT,botafogo,Glorioso,Botafogo
4,100969,Ronald Lopes,-1.1,4,287,True,1,4,Meia,mei,287,VIT,VIT,vitoria,Vitória,Vitória


In [102]:
dados.isnull().sum()
dados = dados.fillna(0)
dados

,id_x,apelido_x,pontuacao,posicao_id,clube_id,entrou,rodada_id,id_y,nome_x,abreviacao_x,id,nome_y,abreviacao_y,slug,apelido_y,nome_fantasia
0,100652,Yuri Alberto,2.9,5,264,True,1,5,Atacante,ata,264,COR,COR,corinthians,Timão,Corinthians
1,100742,Matheus Henrique,-1.9,4,283,True,1,4,Meia,mei,283,CRU,CRU,cruzeiro,Raposa,Cruzeiro
2,100846,Lucas Esteves,2.8,2,284,True,1,2,Lateral,lat,284,GRE,GRE,gremio,Imortal,Grêmio
3,100898,Vitinho,10.7,2,263,True,1,2,Lateral,lat,263,BOT,BOT,botafogo,Glorioso,Botafogo
4,100969,Ronald Lopes,-1.1,4,287,True,1,4,Meia,mei,287,VIT,VIT,vitoria,Vitória,Vitória
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6405,99248,Bareiro,-3.3,5,356,True,21,5,Atacante,ata,356,FOR,FOR,fortaleza,Leão do Pici,Fortaleza
6406,99267,Pochettino,3.2,4,356,True,21,4,Meia,mei,356,FOR,FOR,fortaleza,Leão do Pici,Fortaleza
6407,99274,Alexander Barboza,0.2,3,263,True,21,3,Zagueiro,zag,263,BOT,BOT,botafogo,Glorioso,Botafogo
6408,99552,Rossi,6.3,1,262,True,21,1,Goleiro,gol,262,FLA,FLA,flamengo,Mengão,Flamengo


In [103]:
dados = dados.merge(
    partidas_formatadas,
    left_on=['rodada_id', 'clube_id'],
    right_on=['rodada_id', 'clube_id'],
    how='left'
)
dados

,id_x,apelido_x,pontuacao,posicao_id,clube_id,entrou,rodada_id,id_y,nome_x,abreviacao_x,id,nome_y,abreviacao_y,slug,apelido_y,nome_fantasia,rival_id,placar,posicao,aproveitamento,dificuldade,aproveitamento_5rounds,aproveitamento_3rounds
0,100652,Yuri Alberto,2.9,5,264,True,1,5,Atacante,ata,264,COR,COR,corinthians,Timão,Corinthians,265,1.0,11,"['e', 'd', 'd', 'v', 'e']",-1,0,0
1,100742,Matheus Henrique,-1.9,4,283,True,1,4,Meia,mei,283,CRU,CRU,cruzeiro,Raposa,Cruzeiro,2305,2.0,2,"['v', 'd', 'e', 'v', 'v']",1,0,1
2,100846,Lucas Esteves,2.8,2,284,True,1,2,Lateral,lat,284,GRE,GRE,gremio,Imortal,Grêmio,282,2.0,14,"['d', 'd', 'v', 'e', 'e']",1,0,0
3,100898,Vitinho,10.7,2,263,True,1,2,Lateral,lat,263,BOT,BOT,botafogo,Glorioso,Botafogo,275,0.0,5,"['d', 'v', 'd', 'v', 'v']",-1,0,0
4,100969,Ronald Lopes,-1.1,4,287,True,1,4,Meia,mei,287,VIT,VIT,vitoria,Vitória,Vitória,286,0.0,18,"['e', 'd', 'e', 'd', 'e']",-1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6405,99248,Bareiro,-3.3,5,356,True,21,5,Atacante,ata,356,FOR,FOR,fortaleza,Leão do Pici,Fortaleza,2305,0.0,19,"['e', 'd', 'd', 'd', 'e']",-1,-1,-1
6406,99267,Pochettino,3.2,4,356,True,21,4,Meia,mei,356,FOR,FOR,fortaleza,Leão do Pici,Fortaleza,2305,0.0,19,"['e', 'd', 'd', 'd', 'e']",-1,-1,-1
6407,99274,Alexander Barboza,0.2,3,263,True,21,3,Zagueiro,zag,263,BOT,BOT,botafogo,Glorioso,Botafogo,286,3.0,5,"['d', 'v', 'd', 'v', 'v']",1,0,0
6408,99552,Rossi,6.3,1,262,True,21,1,Goleiro,gol,262,FLA,FLA,flamengo,Mengão,Flamengo,287,8.0,1,"['e', 'v', 'v', 'v', 'e']",1,1,1


In [106]:
dados = dados.drop(columns=["abreviacao_x", "abreviacao_y", "slug", "apelido_y", "nome_fantasia", "entrou", 'id_x', 'posicao_id', 'clube_id', 'rival_id', 'placar', 'posicao', 'aproveitamento'])
dados = dados.rename({"nome_x": "posicao", "nome_y": "time", "apelido_x": "Nome"}, axis=1)
dados

,Nome,pontuacao,rodada_id,id_y,posicao,id,time,dificuldade,aproveitamento_5rounds,aproveitamento_3rounds
0,Yuri Alberto,2.9,1,5,Atacante,264,COR,-1,0,0
1,Matheus Henrique,-1.9,1,4,Meia,283,CRU,1,0,1
2,Lucas Esteves,2.8,1,2,Lateral,284,GRE,1,0,0
3,Vitinho,10.7,1,2,Lateral,263,BOT,-1,0,0
4,Ronald Lopes,-1.1,1,4,Meia,287,VIT,-1,0,0
...,...,...,...,...,...,...,...,...,...,...
6405,Bareiro,-3.3,21,5,Atacante,356,FOR,-1,-1,-1
6406,Pochettino,3.2,21,4,Meia,356,FOR,-1,-1,-1
6407,Alexander Barboza,0.2,21,3,Zagueiro,263,BOT,1,0,0
6408,Rossi,6.3,21,1,Goleiro,262,FLA,1,1,1


In [107]:
dados['mitou'] = dados['pontuacao'] > 7.27
dados

,Nome,pontuacao,rodada_id,id_y,posicao,id,time,dificuldade,aproveitamento_5rounds,aproveitamento_3rounds,mitou
0,Yuri Alberto,2.9,1,5,Atacante,264,COR,-1,0,0,False
1,Matheus Henrique,-1.9,1,4,Meia,283,CRU,1,0,1,False
2,Lucas Esteves,2.8,1,2,Lateral,284,GRE,1,0,0,False
3,Vitinho,10.7,1,2,Lateral,263,BOT,-1,0,0,True
4,Ronald Lopes,-1.1,1,4,Meia,287,VIT,-1,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...
6405,Bareiro,-3.3,21,5,Atacante,356,FOR,-1,-1,-1,False
6406,Pochettino,3.2,21,4,Meia,356,FOR,-1,-1,-1,False
6407,Alexander Barboza,0.2,21,3,Zagueiro,263,BOT,1,0,0,False
6408,Rossi,6.3,21,1,Goleiro,262,FLA,1,1,1,False


In [ ]:
dados.tail(20)

,Nome,pontuacao,rodada_id,id_y,posicao,id,time,dificuldade,aproveitamento_5rounds,aproveitamento_3rounds,mitou
6390,Murilo,4.9,21,3,Zagueiro,275,PAL,1,1,1,False
6391,Erick,3.0,21,5,Atacante,287,VIT,-1,0,0,False
6392,Léo Ortiz,4.7,21,3,Zagueiro,262,FLA,1,1,1,False
6393,Vitão,-0.1,21,3,Zagueiro,285,INT,-1,0,-1,False
6394,Pedro Raul,6.2,21,5,Atacante,354,CEA,0,0,0,False
6395,Negueba,4.7,21,5,Atacante,2305,MIR,1,0,0,False
6396,Jean Lucas,4.4,21,4,Meia,265,BAH,1,0,1,False
6397,Samuel Lino,33.1,21,5,Atacante,262,FLA,1,1,1,True
6398,Viña,6.4,21,2,Lateral,262,FLA,1,1,1,False
6399,Lourenço,2.8,21,4,Meia,354,CEA,0,0,0,False


In [114]:
dados['media_acumulada'] = (
    dados.groupby('Nome')['pontuacao']
      .apply(lambda x: x.shift().expanding().mean())
)



TypeError: incompatible index of inserted column with frame index